In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# 设置请求头部信息，模拟浏览器请求
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
# 设置YachtWorld网站上的搜索条件
search_params = {
    'class': 'sail',
    # 'price': 'USD100000-500000',
    # 'length': '40-50ft'
}
base_url = 'https://www.yachtworld.com'

requests.get(base_url, 
    headers=headers)

<Response [403]>

In [2]:
res = requests.get('https://api-gateway.boats.com/api-yachtworld/search?apikey=827d6e453fef4ce390afe017b5a7db4b&multiFacetedBoatTypeClass=["sail"]&uom=ft&locale=en_US&currency=USD&sort=price-desc&page=1&variant=0&seoContentType=type&videoType=type&isMultiSearch=true&fields=mappedURL,tags,date.created,date.indexed,make,model,condition,boat.boatType,boat.class,year,boat.specifications.dimensions.lengths.nominal,price.type.amount,price.discount,owner.logos,owner.id,owner.name,media.0,id,aliases.imt,aliases.yachtworld,mediaCount,status,isCurrentModel,isOemModel,location.city,location.normalizedCity,location.countrySubDivisionCode,location.isoSubDivisionCode,location.countryCode,location.postalCode,boat.attributes,boat.cpybLogo,salesRep.certs', headers=headers)
result = res.json()

In [3]:
result.keys()
result['search'].keys()
# result['search']['count']
records = result['search']['records']
len(records)
link = records[0]['mappedURL']

## 探索性分析每一个具体url的页面结构

In [8]:
#BoatDetails > div.content.nav-slide.with-sticky-contact.home-block > div.boat-details > div.body > div.boat-details-content > div.details
link = 'https://www.yachtworld.com/yacht/2009-royal-huisman-j-class-yacht-6750313/'
res = requests.get(link, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')
soup.text

"\n\nERROR: The request could not be satisfied\n\n403 ERROR\nThe request could not be satisfied.\n\nRequest blocked.\nWe can't connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.\n\nIf you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.\n\n\n\nGenerated by cloudfront (CloudFront)\nRequest ID: M0n58Tti_mV1Da9MB7Y9ZOmnFLVllzYns-z6gNnIQz2ogA0ywX1NIw==\n\n\n\n"

In [6]:
detail = soup.select_one('#BoatDetails > div.content.nav-slide.with-sticky-contact.home-block > div.boat-details > div.body > div.boat-details-content > div.details')
detail.text

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
items = detail.select('div.header')
len(items)
items[0].text
items[0].next_sibling
for i in range(len(items)):
    if items[i].text.lower()=='specifications':
        break
i
# items[i].text

6

In [ ]:
def kv_attributes(basic, name='Basic'):
    titles = [f'{name}.{"".join(map(lambda x:x.capitalize(),t.text.strip().split()))}' for t in basic.select('td.datatable-title')]
    values = [t.text.strip() for t in basic.select('td.datatable-value')]
    return {k:v for k,v in zip(titles, values)}
kv_attributes(items[0].next_sibling)

{'Basic.Year': '2009',
 'Basic.Make': 'Royal Huisman',
 'Basic.Model': 'J Class Yacht',
 'Basic.Class': 'Racer',
 'Basic.Length': '138.12ft',
 'Basic.FuelType': 'Diesel',
 'Basic.HullMaterial': 'Aluminum',
 'Basic.OfferedBy': 'Burgess London'}

In [ ]:
def specification_attributes(specification, name='Specification'):
    sub_categories = specification.select('div.datatable-category')
    titles = []
    values = []
    for category in sub_categories:
        cate_name = category.find(attrs='sub-title').text
        cate_name = ''.join(map(lambda x:x.capitalize(), cate_name.split()))
        titles += [f'{name}.{cate_name}.{"".join(map(lambda x:x.capitalize(),t.text.strip().split()))}' for t in category.select('td.datatable-title')]
        values += [t.text.strip() for t in category.select('td.datatable-value')]
    return {k:v for k,v in zip(titles, values)}
specification_attributes(items[4].next_sibling, 'Features')

{'Features.ElectricalEquipment.ShorePowerInlet': '✓',
 'Features.ElectricalEquipment.Generator': '✓',
 'Features.Electronics.Depthsounder': '✓',
 'Features.Electronics.Radar': '✓',
 'Features.Electronics.Log-speedometer': '✓',
 'Features.Electronics.RadarDetector': '✓',
 'Features.Electronics.WindSpeedAndDirection': '✓',
 'Features.Electronics.Computer': '✓',
 'Features.Electronics.Repeater(s)': '✓',
 'Features.Electronics.TvSet': '✓',
 'Features.Electronics.NavigationCenter': '✓',
 'Features.Electronics.Plotter': '✓',
 'Features.Electronics.DvdPlayer': '✓',
 'Features.Electronics.Autopilot': '✓',
 'Features.Electronics.Radio': '✓',
 'Features.Electronics.Compas': '✓',
 'Features.Electronics.Gps': '✓',
 'Features.Electronics.CockpitSpeakers': '✓',
 'Features.Electronics.Vhf': '✓',
 'Features.InsideEquipment.Dishwasher': '✓',
 'Features.InsideEquipment.BowThruster': '✓',
 'Features.InsideEquipment.WashingMachine': '✓',
 'Features.InsideEquipment.ElectricBilgePump': '✓',
 'Features.Insid

In [ ]:
def get_price(soup):

    price = soup.select_one('#BoatDetails > div.content.nav-slide.with-sticky-contact.home-block > div.boat-details > div.body > div > div.summary > div > span.payment-total')

    # price.text.strip().split('$')[1]
    x = float("".join(list(filter(str.isdigit,price.text.split('US$')[1]))))
    return x
get_price(soup)

14900000.0

In [ ]:
sn = requests.Session()

In [ ]:
# 爬取所有帆船的year属性和价格数据，并将它们存储到CSV文件中
import warnings


def get_yacht_data(url):
    # res = requests.get(url, headers=headers)
    res = sn.get(url, headers=headers)
    return get_yacht_data_req_res(res)
    
def get_yacht_data_req_res(req_res):
    soup = BeautifulSoup(req_res.text, 'html.parser')
    detail = soup.select_one('#BoatDetails > div.content.nav-slide.with-sticky-contact.home-block > div.boat-details > div.body > div.boat-details-content > div.details')
    assert detail is not None
    items = detail.select('div.header')
    assert len(items)>0
    result = {}
    for i, item in enumerate(items):
            if item.text.lower()=='BASICS'.lower() or item.text.lower()=='PROPULSION'.lower():
                result.update(kv_attributes(items[i].next_sibling), name=item.text.lower().capitalize())
            elif item.text.lower()=='SPECIFICATIONS'.lower() or item.text.lower()=='FEATURES'.lower():
                result.update(specification_attributes(items[i].next_sibling, name=item.text.lower().capitalize()))
    try:
        result['Price'] = get_price(soup)
    except Exception as e:
        # warnings.warn(f"no price for {url}")
        pass
    return result
    
get_yacht_data(link)

{'Basic.Year': '2009',
 'Basic.Make': 'Royal Huisman',
 'Basic.Model': 'J Class Yacht',
 'Basic.Class': 'Racer',
 'Basic.Length': '138.12ft',
 'Basic.FuelType': 'Diesel',
 'Basic.HullMaterial': 'Aluminum',
 'Basic.OfferedBy': 'Burgess London',
 'name': 'Propulsion',
 'Features.ElectricalEquipment.ShorePowerInlet': '✓',
 'Features.ElectricalEquipment.Generator': '✓',
 'Features.Electronics.Depthsounder': '✓',
 'Features.Electronics.Radar': '✓',
 'Features.Electronics.Log-speedometer': '✓',
 'Features.Electronics.RadarDetector': '✓',
 'Features.Electronics.WindSpeedAndDirection': '✓',
 'Features.Electronics.Computer': '✓',
 'Features.Electronics.Repeater(s)': '✓',
 'Features.Electronics.TvSet': '✓',
 'Features.Electronics.NavigationCenter': '✓',
 'Features.Electronics.Plotter': '✓',
 'Features.Electronics.DvdPlayer': '✓',
 'Features.Electronics.Autopilot': '✓',
 'Features.Electronics.Radio': '✓',
 'Features.Electronics.Compas': '✓',
 'Features.Electronics.Gps': '✓',
 'Features.Electronic

In [ ]:
with open('links.txt', 'r') as f:
    links = f.read().split()
links[:5]

['https://www.yachtworld.com/yacht/2023-beneteau-oceanis-46.1----in-stock-8584625/',
 'https://www.yachtworld.com/yacht/2023-jeanneau-sun-odyssey-440-8592771/',
 'https://www.yachtworld.com/yacht/2023-jeanneau-sun-odyssey-490-8592752/',
 'https://www.yachtworld.com/yacht/2023-jeanneau-60-8184733/',
 'https://www.yachtworld.com/yacht/2023-jeanneau-yacht-65-7415679/']

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
from tqdm import tqdm
yacht_data_list = list(range(len(links)))
error_list = {}
def set_data(i):
    # print(i)
    try:
        yacht_data_list[i] = get_yacht_data(links[i])
    except Exception as e:
        error_list[i]=e
        warnings.warn(f"error for {i}: {e}")
    else:
        error_list.pop(i, None)

In [ ]:
# with ThreadPoolExecutor(max_workers=64) as t:
with ThreadPoolExecutor(max_workers=32) as t:
    tasks = [t.submit(set_data, i) for i in range(len(links))]
    # tasks = [t.submit(set_data, i) for i in np.random.choice(len(links), 64, replace=False)]
    # tasks = [t.submit(set_data, i) for i in range(64)]
    for future in tqdm(as_completed(tasks), total=len(tasks)):
        pass    

 41%|████      | 892/2188 [02:24<03:30,  6.17it/s]
/tmp/ipykernel_156073/917640773.py:12: UserWarning: error for 930: 
  warnings.warn(f"error for {i}: {e}")
/tmp/ipykernel_156073/917640773.py:12: UserWarning: error for 931: 
  warnings.warn(f"error for {i}: {e}")
/tmp/ipykernel_156073/917640773.py:12: UserWarning: error for 936: 
  warnings.warn(f"error for {i}: {e}")
/tmp/ipykernel_156073/917640773.py:12: UserWarning: error for 934: 
  warnings.warn(f"error for {i}: {e}")
/tmp/ipykernel_156073/917640773.py:12: UserWarning: error for 940: 
  warnings.warn(f"error for {i}: {e}")
/tmp/ipykernel_156073/917640773.py:12: UserWarning: error for 941: 
  warnings.warn(f"error for {i}: {e}")
/tmp/ipykernel_156073/917640773.py:12: UserWarning: error for 943: 
  warnings.warn(f"error for {i}: {e}")
/tmp/ipykernel_156073/917640773.py:12: UserWarning: error for 944: 
  warnings.warn(f"error for {i}: {e}")
/tmp/ipykernel_156073/917640773.py:12: UserWarning: error for 946: 
  warnings.warn(f"error f

In [ ]:
a = {1:2}
a.pop(1)
a.pop(3, None)
a

{}

In [34]:
error_list # , yacht_data_list
# error_list, yacht_data_list
# links[1890]
# len(error_list)

{}

In [ ]:
with ThreadPoolExecutor(max_workers=32) as t:
    tasks = [t.submit(set_data, k) for k in error_list.keys()]
    # tasks = [t.submit(set_data, i) for i in np.random.choice(len(links), 64, replace=False)]
    # tasks = [t.submit(set_data, i) for i in range(64)]
    for future in tqdm(as_completed(tasks), total=len(tasks)):
        pass   

In [28]:
get_yacht_data(links[1890])

{'Basic.Year': '2022',
 'Basic.Make': 'Bali',
 'Basic.Model': 'Cat Space',
 'Basic.Class': 'Catamaran',
 'Basic.Length': '40.39ft',
 'Basic.FuelType': 'Diesel',
 'Basic.HullMaterial': 'Fiberglass',
 'Basic.HullShape': 'Catamaran',
 'Basic.OfferedBy': 'Rumbo Norte Inversiones Náuticas SL',
 'name': 'Propulsion',
 'Features.ElectricalEquipment.ShorePowerInlet': '✓',
 'Features.ElectricalEquipment.Generator': '✓',
 'Features.ElectricalEquipment.Inverter': '✓',
 'Features.Electronics.Depthsounder': '✓',
 'Features.Electronics.Log-speedometer': '✓',
 'Features.Electronics.WindSpeedAndDirection': '✓',
 'Features.Electronics.Plotter': '✓',
 'Features.Electronics.Autopilot': '✓',
 'Features.Electronics.Radio': '✓',
 'Features.Electronics.Compas': '✓',
 'Features.Electronics.Gps': '✓',
 'Features.Electronics.Vhf': '✓',
 'Features.InsideEquipment.Oven': '✓',
 'Features.InsideEquipment.AirConditioning': '✓',
 'Features.InsideEquipment.ElectricHead': '✓',
 'Features.InsideEquipment.Heating': '✓',


In [ ]:
data_list = filter

In [20]:
df = pd.DataFrame(yacht_data_list)
df.head()

AttributeError: 'int' object has no attribute 'keys'